In [1]:
import pandas as pd
import numpy as np
from exerpy.analyses import _load_json
from exerpy import ExergyAnalysis

In [2]:
def diff_between_simulators(testcase):
    simulator_results = []
    for path in testcase.values():
        contents = _load_json(path)
        if "settings" not in contents:
            contents["settings"] = {}

        simulator_results += [
            ExergyAnalysis.from_json(path, **contents["settings"])
        ]

    sim1 = simulator_results[0]
    sim2 = simulator_results[1]

    columns = ["m", "p", "T"]
    if sim1.chemExLib is not None:
        columns.append("e_CH")
    if sim1.split_physical_exergy:
        columns.append("e_M")
        columns.append("e_T")
    else:
        columns.append("e_PH")

    df_sim1 = pd.DataFrame.from_dict(
        sim1._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")
    df_sim2 = pd.DataFrame.from_dict(
        sim2._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")

    overlapping_index = list(
        set(df_sim1.index.tolist()) & set(df_sim2.index.tolist())
    )
    df_sim1 = df_sim1.loc[overlapping_index].round(6)
    df_sim2 = df_sim2.loc[overlapping_index].round(6)

    # inf means that sim2 has 0 value, comparison does not make sense there
    # and sometimes there seem to be NaN values in the dataframes, those are
    # removed as well
    return (
        (df_sim1 - df_sim2) / df_sim2
    ).abs().replace(np.inf, 0).fillna(0)

In [3]:
testcase = {
    "ebsilon": "cgam_ebs.json",
    "tespy": "cgam_tespy.json"
}
diff_between_simulators(testcase)


,m,p,T,e_CH,e_PH
5,0.000155,0.000268,3.329500e-04,0.003343,3.757031e-04
3,0.000183,0.000052,0.000000e+00,0.000004,5.555921e-04
8,0.000000,0.000000,0.000000e+00,0.000000,5.778811e-09
2,0.000183,0.000000,8.233846e-04,0.000004,1.452651e-03
6,0.000155,0.000296,7.015405e-04,0.003343,1.539015e-03
1,0.000183,0.000000,0.000000e+00,0.000004,0.000000e+00
8P,0.000000,0.000000,0.000000e+00,0.000000,1.265475e-11
10,0.001399,0.000000,0.000000e+00,0.000000,7.728223e-03
9,0.000000,0.000000,2.059617e-09,0.000000,2.902907e-10
7,0.000155,0.000000,1.002368e-03,0.003343,4.903222e-03


In [4]:
testcase = {
    "tespy": "cgam_tespy.json",
    "aspen": "cgam_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_CH,e_PH
5,0.000094,0.000268,9.613326e-05,0.001769,6.715242e-04
3,0.000128,0.000052,0.000000e+00,0.000000,9.748562e-04
8,0.000000,0.000000,0.000000e+00,0.000000,6.687665e-07
2,0.000128,0.000000,2.891440e-04,0.000000,1.153838e-03
6,0.000094,0.000296,9.570306e-05,0.001769,7.732005e-04
1,0.000128,0.000000,0.000000e+00,0.000000,0.000000e+00
8P,0.000000,0.000000,4.781872e-07,0.000000,3.005508e-06
10,0.000724,0.000000,0.000000e+00,0.000000,7.640852e-03
9,0.000000,0.000000,0.000000e+00,0.000000,6.661802e-07
7,0.000094,0.000000,5.023130e-04,0.001769,2.871483e-03


In [5]:
testcase = {
    "ebsilon": "cgam_ebs.json",
    "aspen": "cgam_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_CH,e_PH
5,0.000061,4.094631e-10,2.368488e-04,0.001580,2.955688e-04
3,0.000055,0.000000e+00,0.000000e+00,0.000004,4.187225e-04
8,0.000000,0.000000e+00,0.000000e+00,0.000000,6.629876e-07
2,0.000055,0.000000e+00,5.344786e-04,0.000004,3.004893e-04
6,0.000061,4.221388e-10,6.059046e-04,0.001580,7.670043e-04
1,0.000055,0.000000e+00,0.000000e+00,0.000004,0.000000e+00
8P,0.000000,0.000000e+00,4.781872e-07,0.000000,3.005495e-06
10,0.000675,0.000000e+00,0.000000e+00,0.000000,2.832130e-05
9,0.000000,0.000000e+00,2.059617e-09,0.000000,6.658899e-07
7,0.000061,0.000000e+00,5.005582e-04,0.001580,2.045818e-03


In [6]:
# Load the CSV files
tespy_results = pd.read_csv("cgam_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("cgam_components_ebsilon.csv", index_col=0)

# Step 1: Normalize the component names in Tespy
# Group "drum" and "drum pump" into "EVA"
tespy_results.loc[tespy_results["Component"].isin(["drum"]), "Component"] = "EV"

# Aggregate the values by summing them up
tespy_results = tespy_results.groupby("Component", as_index=False).sum(numeric_only=True)
ebsilon_results = ebsilon_results.groupby("Component", as_index=False).sum(numeric_only=True)

# Step 2: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 3: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 4: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns].copy()
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,AC,2100.488431,2101.221690,0.733260,0.034897
1,APH,2559.563096,2557.498437,-2.064659,-0.080730
2,CC,25314.346057,25397.802218,83.456161,0.328596
3,EV,4586.829577,4591.419972,4.590395,0.099978
4,EXP,2994.122460,2995.010257,0.887797,0.029643
5,PH,1911.387859,1915.951994,4.564135,0.238218
6,TOT,39466.737486,39568.070738,101.333252,0.256099
